# Documentation

This file contain the functions used to serve the user requets.

To understand how we perform the rankin please refer to [this file](https://google.com).

## User story

#### STEP_1: The user uploads a Data_Request

#### Eg.
For example the user wants to retrive the Reviews about Product.
In particluar the user wants to know for each review the DATE and the SCORE of the review. He also wants to know what Product is the review ABOUT and what is the CATEGORY of the product.

#### Review.score | Review.date	| (Review)-[ABOUT]->(Product).name	| (Review)-[ABOUT]->(Product).category |

In order to do so he is asked to upload a Data_Request (DR) where he specifies the:

UOA (Unit-Of-Analysis): this is the item where we are going to start our reserach and should ideally be the Item of most interest in the reseacrh the user is conducting.

ATTRIBUTES

MEASURES

LINKS 

of his request.
This enables us to retrive all the nodes that have the required informations.

#### STEP_2: Identifying the optimal cluster we will scann

Based on the DR we will select the cluster that minimises the number of nodes in each cluster, given our pre-existing raking. To do so we select the NRRT (Node-Relationship-Ranking-Table) of the UOA.
Within the NRRT we look at the ranking feature we have available and we identify the right clustering.

#### Eg.
Given that our user UOA = Review we select the NRRT for Reviews.
In this NRRT we notice that we have only clustered reviws by DATE & [ABOUT]



#### STEP_3:Retrieving the nodes that match the DR
After we have identified the best cluster available (minimize nodes per cluster) we need to match the nodes that satisfy the DR.
In order to do so we need to make sure that all the ATT/MEAS/LINKS requested in the DR are available.
We first make sure that all the features of the UOA are met, and we then move on to scan the linked Items.


### Eg.

for our UOA = Review we need to make sure that all the ATTRIBUTES and MEASURES for each Review are avialable. For the links and the ATT/MEAS/LINK of the other Item linked to the review (in our case Product) this is more complicated.

We scan the ABM of each node in the cluster to see if they have the required features.

When looking at the linked items we need to discriminated between (DIRECT_LINKS vs BEST_RATED_LINK)

DIRECT_LINK:
If you link a (Review) to a (Book) based on the ISBN with a direct link you DON’T INHERIT THE ATTRIBUTES & MEASURES  of the other nodes in the cluster but you DO INHERIT the INCOMING_BEST_RATED LINKS of the cluster you are in. So if for example a DR is asking about the review (summary_score) of the books and for the book they want to know (book_title, book_publisher) if the Review is linked directly to the Book and the Book instance the Review is linked to doesn't have a Book_publisher we cannot look at the book_publisher of another instance in the cluster. However if the book_publisher is not an attribute or measure  but is insted a link then if the link (Review)-[ABOUT]-(Book)-[PUBLISHED_BY]-(Company) the second part of the link [PUBLISHED_BY]-(Company) can be taken from the Book_NRRT incoming link if there is any in the cluster where the book is in.


BEST_RATED_LINK:
If you link a (Review) to a (Book) based on the ISBN with a BEST_RATED link you DO INHERIT THE ATTRIBUTES & MEASURES  of the best rated nodes in the landing cluster. And as with the direct links you also INHERIT the INCOMING_BEST_RATED LINKS of the cluster you are in
we scan the Review clustered by date and [ABOUT] to selct all the nodes that have a Review.score
we select the nodes that have a score & a ABOUT link pointing to Product
Then we select only the product that have a name & category
Etsy has it Ebay & Amazon is more complex
"If the Ebay Review has a direct link to a product https://ebay.com/Product/123 i need to check if:
  1. the landing node has a Product_name & category
  2. If we also want to know the MANUFACTURER we must check if the link is not defined in the node if there is a link in the cluster the node is in or in the incoming interaction links. "




---------------------------------------------------------------------------------------------------
## Notes

---------------------------------------------------------------------------------------------------


1. from the UOA select the best cluster
2. for each cluster select the nodes that have the ATT & MEAS
3. for each cluster check the (CONFIRMATION/SIMILARITY ranking & the TRUST/STAKING ranking)
4. for links check: (NRRT-Incoming links) + (Direct outgoing links BEST_RATED) + (OUTGOING DIRECT links) + (INCOMING DIRECT interaction links)
    1. for NRRT-incoming links verifify if the pointing node meets the criteria
    2. for Direct outgoing links verify if the node they are poing to meet the criteria ATT/MEAS + for links check (outgoing links (direct/best_rated) + NRRT-Incoming links fo the cluster the node is in + 


In [1]:
'''
---------------------------------------------------------------
All imports here
---------------------------------------------------------------
'''

import json
import pandas as pd
import re
import urllib.request

In [2]:
'''
---------------------------------------------------------------
All the data.files imported here
---------------------------------------------------------------
'''

# Data_Request
with urllib.request.urlopen('https://raw.githubusercontent.com/mister-one/onesto/master/data_request/dr_test_1') as url:
    dr = json.loads(url.read().decode())

dr_json_str = json.dumps(dr,indent=2, sort_keys=False)
print(dr_json_str)

{
  "UOA": "Review",
  "Review": {
    "feature_1": "date",
    "feature_2": "score",
    "feature_3": "-[ABOUT]->(Product)"
  },
  "Product": {
    "feature_1": "name",
    "feature_2": "category"
  }
}


In [35]:
'''
ALL FUNCTIONS
'''

'''
ocf (Optimal Cluster Finder)
This function is used to find the right cluster when retriving the data
For each item we look at the features in the DR and find the item that has the lowest number of nodes.
'''
def ocf(data_request):
    # --------
    # INPUTS
    items =  []
    nrrt = {}
    features_by_item = {}
    unique_ranking_features = {}
    optimal_cluster = {}
    
    # --------
    # PROCESS
    
    #STEP_1: retrive the items from the data_request
    for x in data_request:
        if x != 'UOA':
            items.append(x)
    
    #STEP_2: retrive the items NRRT from the data_request
    for x in items:
        nrrt[x]=(f"https://raw.githubusercontent.com/mister-one/onesto/master/nrrt/{x}")
    
    
    #STEP_3 dictionary of the features by item in the request
    for x in data_request:
        if x != 'UOA':
            for y in data_request[x]:
                if x in features_by_item:
                    features_by_item[x].append(data_request[x][y])
                else:
                    features_by_item[x] = [data_request[x][y]]
    
    # STEP_4 check the # of ranking feature available for each item
    for key in nrrt:
        with urllib.request.urlopen(f'{nrrt[key]}') as url:
            data = json.loads(url.read().decode())
            for x in data:
                for y in data[x]:
                    urf = set()
                    for z in y['rf']:
                        urf.add(z['feature_name'])
                unique_ranking_features[x] = list(urf)
                
    # STEP_5 identify the cluster we can use for each item
    for x in features_by_item:
        features = []
        for y in features_by_item[x]:
            if y in unique_ranking_features[x]:
                features.append(y)
        optimal_cluster[x]=features
  
    
    
    
    # --------
    # OUTPUT
    return (optimal_cluster)
    # items =  [list of Items in the data_request (Car, Review, Engine)]
    # nrrt = [list of http adress of the ranking table]
    # features_by_item = {Dictionay of the features for each item of the DR}
    # available_ranking_features = {list of columns in the NRRT}

    


'''
Select Cluster
This function returns the NRRT_ID of the clusters that match the right LOG
'''    
def select_cluster(data_request):
    # --------
    # INPUTS
    uoa =  data_request["UOA"] #find the UOA in the data_request (EG. Review)
    optimal_cluster = ocf(data_request)
    # --------
    # PROCESS
    
    #STEP_1: retrive the right cluster in the UOA
    selected_clusters = {}
    for x in optimal_cluster:
        with urllib.request.urlopen(f"https://raw.githubusercontent.com/mister-one/onesto/master/nrrt/{x}") as url:
            nrrt = json.loads(url.read().decode())
        selected_cluster_id = []
        for y in nrrt[x]:
            ls =[]
            for z in y['rf']:
                ls.append(z['feature_name'])
            if all(item in optimal_cluster[x] for item in ls):
                selected_cluster_id.append(y['NRRT_ID'])
        selected_clusters[x]=selected_cluster_id
        
        
    # --------
    # OUTPUT
    return(selected_clusters)

'''
Select Cluster
This function returns the NRRT_ID of the clusters that match the right LOG
is used to find the nodes that match the Data_Request requiremnts.
After we have identified the best cluster we have avilable we need to:
1. Verify all ATTRIBUTES are present for the nodes in each different cluster
2. Verify all MEASURES are present for the nodes in each different cluster
3. Verify the LINKS are present for the nodes in each different cluster
4. Make sure we check the CONFIRMATION/SIMILARITY ranking & the TRUST/STAKING ranking within each cluster
'''   


'\nSelect Cluster\nThis function returns the NRRT_ID of the clusters that match the right LOG\nis used to find the nodes that match the Data_Request requiremnts.\nAfter we have identified the best cluster we have avilable we need to:\n1. Verify all ATTRIBUTES are present for the nodes in each different cluster\n2. Verify all MEASURES are present for the nodes in each different cluster\n3. Verify the LINKS are present for the nodes in each different cluster\n4. Make sure we check the CONFIRMATION/SIMILARITY ranking & the TRUST/STAKING ranking within each cluster\n'

In [44]:
# ------------------------------------------------------------
# ------------------------------------------------------------
# Test
# ------------------------------------------------------------
# ------------------------------------------------------------

def select_cluster(data_request):
    # --------
    # INPUTS
    uoa =  data_request["UOA"] #find the UOA in the data_request (EG. Review)
    optimal_cluster = ocf(data_request)
    # --------
    # PROCESS
    
    #STEP_1: retrive the right cluster in the UOA
    selected_clusters = {}
    for x in optimal_cluster:
        with urllib.request.urlopen(f"https://raw.githubusercontent.com/mister-one/onesto/master/nrrt/{x}") as url:
            nrrt = json.loads(url.read().decode())
        selected_cluster_id = []
        for y in nrrt[x]:
            ls =[]
            for z in y['rf']:
                ls.append(z['feature_name'])
        print(x)
        print(optimal_cluster)
        print(ls)



In [45]:
select_cluster(dr)

Review
{'Review': ['date', 'score'], 'Product': []}
['score', 'date', '[ABOUT]->(Product)']
Product
{'Review': ['date', 'score'], 'Product': []}
['score', 'date', '[ABOUT]->(Product)']


In [ ]:
 if all(item in optimal_cluster[x] for item in ls):
                selected_cluster_id.append(y['NRRT_ID'])
        selected_clusters[x]=selected_cluster_id
        
        
    # --------
    # OUTPUT
    return(selected_clusters)

---------------------------------------------------------------------------------------------------
## Notes

---------------------------------------------------------------------------------------------------

1. from the UOA select the best cluster
2. for each cluster select the nodes that have the ATT & MEAS
3. for each cluster check the (CONFIRMATION/SIMILARITY ranking & the TRUST/STAKING ranking)
4. for links check: (NRRT-Incoming links) + (Direct outgoing links BEST_RATED) + (OUTGOING DIRECT links) + (INCOMING DIRECT interaction links)
    1. for NRRT-incoming links verifify if the pointing node meets the criteria
    2. for Direct outgoing links verify if the node they are poing to meet the criteria ATT/MEAS + for links check (outgoing links (direct/best_rated) + NRRT-Incoming links fo the cluster the node is in + 


In [36]:
select_cluster(dr)

{'Review': [], 'Product': []}

In [24]:
matched_clusters = {}
features_by_item = {'Review': ['date', 'score', '-[ABOUT]->(Product)'],'Product': ['name', 'category']}
unique_ranking_features = {'Review': ['[ABOUT]->(Product)', 'date', 'score'], 'Product': ['[ABOUT]->(Product)', 'date', 'score']}

for x in features_by_item:
    features = []
    for y in features_by_item[x]:
        if y in unique_ranking_features[x]:
            features.append(y)
    matched_clusters[x]=features



{'Review': ['date', 'score'], 'Product': []}


In [200]:
copy_dr = json.loads(dr)

#print(copy_dr["UOA"])
'''
for x in copy_dr:
    if x == copy_dr['UOA']:
        for y in copy_dr[x]:
            print(copy_dr[x][y])'''

print(copy_dr)
for x in copy_dr:
        if x != 'UOA':
            print(x)

{'oscar_age_female.csv': {'feature_1': 'date', 'feature_2': 'score', 'feature_3': '-[ABOUT]->(Product)'}, 'hurricanes.csv': {'feature_1': 'date', 'feature_2': 'score', 'feature_3': '-[ABOUT]->(Product)'}, 'grades.csv': {'feature_1': 'name', 'feature_2': 'category'}}
oscar_age_female.csv
hurricanes.csv
grades.csv


In [ ]:
matched_clusters = []
        for x in data['Review']:
            c = []
    for y in x['rf']:
        f.append(y['feature_name'])
    print(all(x in optimal_cluster for x in c))
    matched_clusters.append(x['NRRT_ID'])

In [33]:
print(dr_json_str)

{
  "UOA": "Review",
  "Review": {
    "feature_1": "date",
    "feature_2": "score",
    "feature_3": "-[ABOUT]->(Product)"
  },
  "Product": {
    "feature_1": "name",
    "feature_2": "category"
  }
}


In [ ]:
'''
1. User uploads a data_request(DR)
2. select all the instances that have the data requested:
    2.1 identify the optimal cluster by looking at the NRRT and find the greatest number of ranking features 
        compatible with the DR. Starting from the UOA=Review we look at the NRRT/Review and ideally
        we would have a ranking by distinct (date,score and -[ABOUT]->(Product))
    2.2 select the cluster ids that are 
3. return all the instances that match the results


selecting the clusters
1. select the NRRT for the UOA
2. Starting from the NRRT ranking features if all the NRRT features are in the DR
    then select the instances

----------------------------------------------------
data_Request
----------------------------------------------------

"Review": {
    "feature_1": "date",
    "feature_2": "score",
    "feature_3": "-[ABOUT]->(Product)"

----------------------------------------------------
NRRT
https://onesto.world/NRRT/Review
----------------------------------------------------
"Review": [
    {
      "NRRT_ID": "1",
      "rf": [
        {
          "feature_name": "score",
          "feature_type": "MEAS",
          "feature_value": "3/5"
        },
        {
          "feature_name": "date",
          "feature_type": "MEAS",
          "feature_value": "January 2020"
        },
        {
          "feature_name": "[ABOUT]->(Product)",
          "feature_type": "LINK",
          "feature_value": "Product/123123"
        }
      ],
      "num_node_available": 2,
      "node_ranking": [
        {
          "node_address": "https://gooe.com/Review/1",
          "node_abm": "https://raw.githubusercontent.com/mister-one/onesto/master/ABM/Review/1",
          "trust_score": 0.1,
          "frequency_score": 1
        },
        {
          "node_address": "https://gooe.com/Review/2",
          "node_abm": "https://raw.githubusercontent.com/mister-one/onesto/master/ABM/Review/1",
          "trust_score": 0.1,
          "frequency_score": 1
        }
      ],
      "link_ranking": [
        {
          "origin": "Allert",
          "incoming_node_address": [
            "https://hello.com/Alert/123",
            "https://hello.com/Alert/123",
            "https://hello.com/Alert/123",
            "https://hello.com/Alert/123"
          ]
        }
      ]
    }
    
scansioni le riview e guardi gli attributi e misure perche il link potrebbe venire da qalcun altro
attributes and measures must be there perche se non ci sono no li puoi ereditare.

se un istanza non

In the nrrt i have the rf {model_name, [produced_by]-(Company)}
the DR want the Car{model_name, production_year,[produced_by]-(Company)}
1. if all the RF match the DR then return the NNRT/id for the one that match
2. If the Rf availabe are less than
3. if the rf availabe are more return just the one that have a perfect matah
Need to select the max number of ranking features
for each NRRT cluster, starting from the UOA NRRT, we check if all the RF in the NRRT match the DR:
CASE_1
All the RF in the NRRT match the DR, then there is PERFECT_MATCH
RF = 4 (name,surname,height,weight), DR = 4 (name,surname,height,weight)

CASE_2
if the RF in the NRRT cluster is more granular (more features) than the features in the DR then is going to return false
RF = 4 (name,surname,height,weight), DR = 3 (name,surname,height)

CASE_3
if we have less granular rf in the NRRT then its going to return TRUE
RF = 4 (name,surname,height,weight), DR = 5 (name,surname,height,weight,sex)
however the issue is that is going to return true also for the clusters with 2 RF. So we always need to select
only the cluster_id where (all_RF in DR =TRUE) and among the TRUE one where count(rf) = max (RF count)
in this case
'''
# function to load the DR
# function to identify the OCF (all RF in the NRRT_cluster_id are in the DR and take the max number of RF)
# define function to select the right instances based on score LINK can be inherited (ATTR & MEAS NOOOOOO)
    # make sure (all ATTR & MEAS of the instances match DR) but LINK can be inherited
    # eg in the NRRT we only cluster instances of Cars by MODEL & YEAR 
        # if a DR wants  {car.model, car.year, car.[produced_by]->(Company.name)
        # the instances must have all the ATTR & MEAS but they can miss an outgoing link to Company.
        # Indeed the instance can derive the connection from the incoming_links_ranking section of the NRRT
        
'''
Nrrt cluster by Car{model_name,year}
dr {car.model_name, car.year, car.[produced_by]->(Company.name),car.[produced_by]->(Company.foundation_year)}
'''
'''
1. find the cluster
2. for the UOA and cluster make sure that all the instances have the ATTR & MEAS & LINKS
    if they dont have a link then chek in the NRRT link for the UOA
    2.1 then move onto the next Item for example company:
            if the link is Direct then it must have all the required info
            if the link is best rated then check if in the landing Company cluter there is an instance that has the required data
            
'''